<a href="https://colab.research.google.com/github/somitrasingh/deeplearning/blob/main/Next_Word_Predictor_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [3]:
import spacy

In [4]:
nlp = spacy.blank('en')
nlp.add_pipe('sentencizer')

In [5]:
nlp.analyze_pipes()

{'summary': {'sentencizer': {'assigns': ['token.is_sent_start', 'doc.sents'],
   'requires': [],
   'scores': ['sents_f', 'sents_p', 'sents_r'],
   'retokenizes': False}},
 'problems': {'sentencizer': []},
 'attrs': {'doc.sents': {'assigns': ['sentencizer'], 'requires': []},
  'token.is_sent_start': {'assigns': ['sentencizer'], 'requires': []}}}

In [6]:
doc = nlp(document.lower())

In [7]:
nlp.analyze_pipes()

{'summary': {'sentencizer': {'assigns': ['token.is_sent_start', 'doc.sents'],
   'requires': [],
   'scores': ['sents_f', 'sents_p', 'sents_r'],
   'retokenizes': False}},
 'problems': {'sentencizer': []},
 'attrs': {'doc.sents': {'assigns': ['sentencizer'], 'requires': []},
  'token.is_sent_start': {'assigns': ['sentencizer'], 'requires': []}}}

In [8]:
vocab = {"<UNK>" : 0}

for token in doc:
  if token.text not in vocab:
    vocab[token.text] = len(vocab)

print(vocab)
len(vocab)

{'<UNK>': 0, 'about': 1, 'the': 2, 'program': 3, '\n': 4, 'what': 5, 'is': 6, 'course': 7, 'fee': 8, 'for': 9, ' ': 10, 'data': 11, 'science': 12, 'mentorship': 13, '(': 14, 'dsmp': 15, '2023': 16, ')': 17, 'follows': 18, 'a': 19, 'monthly': 20, 'subscription': 21, 'model': 22, 'where': 23, 'you': 24, 'have': 25, 'to': 26, 'make': 27, 'payments': 28, 'of': 29, 'rs': 30, '799': 31, '/': 32, 'month': 33, '.': 34, 'total': 35, 'duration': 36, '?': 37, '7': 38, 'months': 39, 'so': 40, 'becomes': 41, '*': 42, '=': 43, '5600(approx': 44, 'syllabus': 45, 'we': 46, 'will': 47, 'be': 48, 'covering': 49, 'following': 50, 'modules': 51, ':': 52, 'python': 53, 'fundamentals': 54, 'libraries': 55, 'analysis': 56, 'sql': 57, 'maths': 58, 'machine': 59, 'learning': 60, 'ml': 61, 'algorithms': 62, 'practical': 63, 'mlops': 64, 'case': 65, 'studies': 66, 'can': 67, 'check': 68, 'detailed': 69, 'here': 70, '-': 71, 'https://learnwith.campusx.in/courses/campusx-data-science-mentorship-program-637339afe4b

284

In [9]:
sentences = []
for sent in doc.sents:
  indexed = []
  for token in sent:
    indexed.append(vocab[str(token)])
  sentences.append(indexed)

In [10]:
sentences

[[1,
  2,
  3,
  4,
  5,
  6,
  2,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  3,
  14,
  15,
  16,
  17,
  4,
  2,
  7,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  20,
  28,
  29,
  30,
  31,
  32,
  33,
  34],
 [4, 5, 6, 2, 35, 36, 29, 2, 7, 37],
 [4, 2, 35, 36, 29, 2, 7, 6, 38, 39, 34],
 [40, 2, 35, 7, 8, 41, 31, 42, 38, 43, 30, 44, 34, 17],
 [4, 5, 6, 2, 45, 29, 2, 13, 3, 37],
 [4,
  46,
  47,
  48,
  49,
  2,
  50,
  51,
  52,
  4,
  53,
  54,
  4,
  53,
  55,
  9,
  11,
  12,
  4,
  11,
  56,
  4,
  57,
  9,
  11,
  12,
  4,
  58,
  9,
  59,
  60,
  4,
  61,
  62,
  4,
  63,
  61,
  4,
  64,
  4,
  65,
  66,
  4,
  24,
  67,
  68,
  2,
  69,
  45,
  70,
  71,
  72,
  4,
  47,
  73,
  60,
  74,
  75,
  48,
  19,
  76,
  29,
  77,
  3,
  37],
 [4, 78, 79, 75, 74, 73, 60, 80, 81, 82, 19, 76, 29, 77, 3, 83, 84, 34],
 [4, 5, 85, 86, 87, 19, 88, 89, 37],
 [47, 86, 90, 19, 91, 29, 2, 89, 37],
 [4,
  92,
  93,
  94,
  95,
  81,
  96,
  79,
  40,
  97,
  85,
  24,
  87,
  1

In [11]:
data = []
for sent in sentences:
  for i in range(1, len(sent)):
    data.append(sent[:i+1])

data

[[1, 2],
 [1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 2],
 [1, 2, 3, 4, 5, 6, 2, 7],
 [1, 2, 3, 4, 5, 6, 2, 7, 8],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3, 14],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3, 14, 15],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3, 14, 15, 16],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3, 14, 15, 16, 17],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3, 14, 15, 16, 17, 4],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3, 14, 15, 16, 17, 4, 2],
 [1, 2, 3, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 13, 3, 14, 15, 16, 17, 4, 2, 7],
 [1,
  2,
  3,
  4,
  5,
  6,
  2,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  3,
  14,
  15,
  16,
  17,
  4,
  2,
  7,

In [12]:
len(data)

1021

In [13]:
max_len = max(len(seq) for seq in data)


In [14]:
max_len

65

In [15]:
training_data = np.zeros((len(data), max_len))

In [16]:
training_data.shape

(1021, 65)

In [17]:
for i, seq in enumerate(data):
  start = max_len - len(seq)
  training_data[i,start:] = seq

In [18]:
training_data

array([[  0.,   0.,   0., ...,   0.,   1.,   2.],
       [  0.,   0.,   0., ...,   1.,   2.,   3.],
       [  0.,   0.,   0., ...,   2.,   3.,   4.],
       ...,
       [  0.,   0.,   0., ..., 172., 281., 282.],
       [  0.,   0.,   0., ..., 281., 282., 283.],
       [  0.,   0.,   0., ..., 282., 283.,   4.]])

In [19]:
X = training_data[:, :-1]
y = training_data[:, -1]

In [20]:
X = torch.tensor(X, dtype=torch.long)

In [21]:
y = torch.tensor(y, dtype=torch.long)

In [22]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [23]:
class customDataset(Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]


In [24]:
dataset = customDataset(X, y)

In [25]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [26]:
class SimpleLSTM(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, 128)
    self.lstm = nn.LSTM(128, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, input):
    y = self.embeddings(input)
    _, (y, _) = self.lstm(y)
    y = self.fc(y.squeeze(0))

    return y


In [27]:
model = SimpleLSTM(len(vocab))

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [29]:
epochs = 50

In [30]:
for epoch in range(epochs):
  total_loss = 0
  for features, labels in dataloader:
    y = model(features)
    loss = criterion(y, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  print(f'Epoch :{epoch+1}, Loss: {total_loss}')


Epoch :1, Loss: 176.3062891960144
Epoch :2, Loss: 152.93060731887817
Epoch :3, Loss: 135.80773091316223
Epoch :4, Loss: 121.43075180053711
Epoch :5, Loss: 108.64501667022705
Epoch :6, Loss: 97.26595997810364
Epoch :7, Loss: 87.0558180809021
Epoch :8, Loss: 78.14838433265686
Epoch :9, Loss: 69.93571829795837
Epoch :10, Loss: 62.635104298591614
Epoch :11, Loss: 56.18767428398132
Epoch :12, Loss: 50.47537553310394
Epoch :13, Loss: 45.33693742752075
Epoch :14, Loss: 40.56133669614792
Epoch :15, Loss: 36.769768834114075
Epoch :16, Loss: 33.44281029701233
Epoch :17, Loss: 30.40306830406189
Epoch :18, Loss: 27.81668132543564
Epoch :19, Loss: 25.55077964067459
Epoch :20, Loss: 23.561616510152817
Epoch :21, Loss: 21.907729983329773
Epoch :22, Loss: 20.515928506851196
Epoch :23, Loss: 19.17771190404892
Epoch :24, Loss: 17.94706067442894
Epoch :25, Loss: 16.84895959496498
Epoch :26, Loss: 15.991882503032684
Epoch :27, Loss: 15.187187820672989
Epoch :28, Loss: 14.443950295448303
Epoch :29, Loss: 1

In [31]:
model.eval()

SimpleLSTM(
  (embeddings): Embedding(284, 128)
  (lstm): LSTM(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=284, bias=True)
)

In [32]:
def text_to_index(text, vocab):
  doc = nlp(text)
  indexed = []
  for token in doc:
    if token in vocab:
      indexed.append(vocab[token])
    else:
      indexed.append(vocab['<UNK>'])

  return indexed


In [33]:
def format_data(indexed, ):
  data = []
  for i in range(1, len(indexed)):
    data.append(sent[:i+1])

  training_data = np.zeros((len(data), max_len))
  for i, seq in enumerate(data):
    start = max_len - len(seq)
    training_data[i,start:] = seq

  return torch.tensor(training_data, dtype=torch.long)


In [37]:
def predict(model, vocab, text):
  indexed = text_to_index(text, vocab)
  training_seq = format_data(indexed)

  for input in training_seq:
    output = model(input.unsqueeze(0))
    output = nn.functional.softmax(output, dim=1)
    value, index = torch.max(output, dim=1)

  print(list(vocab.keys())[index])


In [45]:
input = "only for the comes under placement assistance \n portfolio building"
predict(model, vocab, input)

sessions


In [36]:
keys_list = list(vocab.keys())